### Importación de librerias

In [ ]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Liberías de optimización
# ==============================================================================
from pulp import *

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

### Importación de datos

In [ ]:
origenes = ["EU", "BRA", "USA", "CA"]  # Nodos de origen


In [51]:
# Leemos los conjuntos
import pandas as pd
import numpy as np
import sys
import os


Origin = ['manizales','medellin','bucaramanga','bogota','ibague','neival','soacha','pereira']

Destination = ['españa','alemania','brazil','panama','mexico','estados_unidos']

CTL = ['ctl_bogota',
       'ctl_pereira',
       'ctl_manizales',
       'ctl_soacha',
       'ctl_neiva',
       'ctl_ibague',
       'ctl_cienaga',
       'ctl_la_dorada',
       'ctl_puerto_berrio']

PCTL = ['ctl_soeldad','ctl_santa_marta','ctl_cartagena']

Port = ['puerto_de_cartagena','puerto_de_barranquilla','puerto_de_santa_marta']

CAPk = pd.read_excel("model_data.xlsx",sheet_name="CAPk").set_index("CTL")
CAPm = pd.read_excel("model_data.xlsx",sheet_name="CAPm").set_index("PCTL")

Fk = pd.read_excel("model_data.xlsx",sheet_name="Fk")#.set_index("CTL")
Fm = pd.read_excel("model_data.xlsx",sheet_name="Fm")#.set_index("PCTL")

Wij=pd.read_excel("model_data.xlsx",sheet_name="Wij")#.set_index(['Origin', 'Destination'])

costs=pd.read_excel("model_data.xlsx",sheet_name="cost")#.set_index(['code_inicio', 'code_fin'], inplace=True)


### Transformación de datos

In [29]:
#Demanda en formato dict

origenes = Wij.Origin.unique()
destinos = Wij.Destination.unique()
DEMANDA_DICT = dict()
type_dict = 'records'

for origen in origenes:
  DEMANDA_DICT[origen] = {}
  for destino in destinos:
    valor = Wij[Wij.Origin.isin([origen]) & Wij.Destination.isin([destino])]
    valor.reset_index(inplace=True, drop=True)
    DEMANDA_DICT[origen][destino] = valor.loc[0, 'Wij']

print(DEMANDA_DICT)

{'manizales': {'españa': 304, 'alemania': 293, 'brazil': 336, 'panama': 199, 'mexico': 269, 'estados_unidos': 468}, 'medellin': {'españa': 58, 'alemania': 469, 'brazil': 100, 'panama': 114, 'mexico': 440, 'estados_unidos': 165}, 'bucaramanga': {'españa': 18, 'alemania': 0, 'brazil': 264, 'panama': 199, 'mexico': 148, 'estados_unidos': 25}, 'bogota': {'españa': 371, 'alemania': 87, 'brazil': 120, 'panama': 71, 'mexico': 350, 'estados_unidos': 65}, 'ibague': {'españa': 187, 'alemania': 282, 'brazil': 237, 'panama': 89, 'mexico': 261, 'estados_unidos': 44}, 'neival': {'españa': 51, 'alemania': 489, 'brazil': 32, 'panama': 75, 'mexico': 214, 'estados_unidos': 453}, 'soacha': {'españa': 43, 'alemania': 240, 'brazil': 181, 'panama': 439, 'mexico': 266, 'estados_unidos': 361}, 'pereira': {'españa': 262, 'alemania': 218, 'brazil': 466, 'panama': 400, 'mexico': 378, 'estados_unidos': 359}}


In [47]:
#Costos en formato dict

#Costos fijos Fk
ctls = Fk.CTL.unique()
Fk_dict = dict()

for trasbordo in ctls:
  valor = Fk[Fk.CTL.isin([trasbordo])]
  valor.reset_index(inplace=True, drop=True)
  Fk_dict[trasbordo] = valor.loc[0, 'Fk']

print(Fk_dict)

#Costos fijos Fm
pctls = Fm.PCTL.unique()
Fm_dict = dict()

for trasbordo in pctls:
  valor = Fm[Fm.PCTL.isin([trasbordo])]
  valor.reset_index(inplace=True, drop=True)
  Fm_dict[trasbordo] = valor.loc[0, 'Fm']

print(Fm_dict)

{'ctl_bogota': 37553.69934640523, 'ctl_pereira': 36382.836601307186, 'ctl_manizales': 36358.875816993466, 'ctl_soacha': 36915.27450980392, 'ctl_neiva': 35896.504575163395, 'ctl_ibague': 36119.61568627451, 'ctl_cienaga': 36642.69281045752, 'ctl_la_dorada': 36307.18954248366, 'ctl_puerto_berrio': 35981.56862745098}
{'ctl_cartagena': 37758.67973856209, 'ctl_soeldad': 36933.921568627455, 'ctl_santa_marta': 39106.73202614379}


In [ ]:
#Costos en formato dict

#Costos de transporte Cijkmp


In [ ]:
#param Cijp{i in Origin, j in Destination, p in PCTL};##Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío directo carretero al puerto p
#param Cijkp{i in Origin, j in Destination, k in CTL, p in Port};##Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal k ∈ K al puerto p
#param Cijmp{i in Origin, j in Destination, m in PCTL, p in Port};##Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal m ∈ M al puerto p    
#param Cijkmp{i in Origin, j in Destination,  k in CTL, m in PCTL, p in Port};#Costo de transporte unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío a través del par de terminales k ∈ K y m ∈ M al puerto p

#param Eijp{i in Origin, j in Destination, p in PCTL};##Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío directo carretero al puerto p
#param Eijkp{i in Origin, j in Destination, k in CTL, p in Port};##Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal k ∈ K al puerto p
#param Eijmp{i in Origin, j in Destination, m in PCTL, p in Port};##Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativade envío a través de la terminal m ∈ M al puerto p    
#param Eijkmp{i in Origin, j in Destination,  k in CTL, m in PCTL, p in Port};#Costo de externalidad unitario del origen i ∈ I al destino j ∈ J utilizando la alternativa de envío a través del par de terminales k ∈ K y m ∈ M al puerto p
    